In [80]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import cifar10
from tensorflow.keras import layers
from keras.layers import ZeroPadding2D
import tensorflow as tf


import matplotlib.pyplot as plt

# 데이터셋 생성하기(정규화)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

# 원-핫 인코딩
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

(x_train, x_valid) = x_train[5000:], x_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000]

In [81]:
#conv_base=tf.keras.applications.ResNet50(weights='imagenet',include_top=False)
from keras.applications import VGG19
conv_base = VGG19(include_top=False,weights='imagenet',input_shape=(32,32,3),classes=y_train.shape[1])

In [82]:
conv_base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [83]:
from keras import Sequential
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout

model_1= Sequential()
model_1.add(conv_base) #Adds the base model (in this case vgg19 to model_1)
model_1.add(Flatten()) 
model_1.add(Dense(1024,activation=('relu'),input_dim=512))
model_1.add(Dropout(0.25))
model_1.add(Dense(512,activation=('relu'))) 
model_1.add(Dropout(0.25))
model_1.add(Dense(256,activation=('relu'))) 
#model_1.add(Dropout(.3))#Adding a dropout layer that will randomly drop 30% of the weights
model_1.add(Dense(128,activation=('relu')))
#model_1.add(Dropout(.2))
model_1.add(Dense(10,activation=('softmax')))

In [84]:
from keras.optimizers import SGD,Adam
learn_rate=.001

sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)
adam=Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model_1.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])

In [85]:
conv_base.trainable = True

In [86]:
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block4_conv1':
        print("right")  
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

right


In [87]:
history= model_1.fit(x_train, y_train, epochs = 40, batch_size = 100, validation_data = (x_valid, y_valid))


Epoch 1/40
450/450 [==============================] - 28s 63ms/step - loss: 1.4264 - accuracy: 0.4855 - val_loss: 0.8776 - val_accuracy: 0.6950
Epoch 2/40
450/450 [==============================] - 28s 62ms/step - loss: 0.8091 - accuracy: 0.7252 - val_loss: 0.6987 - val_accuracy: 0.7644
Epoch 3/40
450/450 [==============================] - 28s 61ms/step - loss: 0.6579 - accuracy: 0.7770 - val_loss: 0.5982 - val_accuracy: 0.7984
Epoch 4/40
450/450 [==============================] - 28s 62ms/step - loss: 0.5574 - accuracy: 0.8121 - val_loss: 0.5835 - val_accuracy: 0.8040
Epoch 5/40
450/450 [==============================] - 28s 62ms/step - loss: 0.4886 - accuracy: 0.8334 - val_loss: 0.5379 - val_accuracy: 0.8152
Epoch 6/40
450/450 [==============================] - 28s 62ms/step - loss: 0.4229 - accuracy: 0.8561 - val_loss: 0.5147 - val_accuracy: 0.8258
Epoch 7/40
450/450 [==============================] - 28s 62ms/step - loss: 0.3654 - accuracy: 0.8756 - val_loss: 0.4572 - val_accuracy:

In [92]:
print('## training loss ##')
print(history.history['loss'])


result = model_1.evaluate(x_test, y_test, batch_size=100)
print('##  loss and accuracy ##')
print(result)

## training loss ##
[1.4264147281646729, 0.8091477155685425, 0.6578518152236938, 0.5574468970298767, 0.4886305034160614, 0.4229220151901245, 0.3653644025325775, 0.31619465351104736, 0.2686309814453125, 0.23194576799869537, 0.19415177404880524, 0.16180050373077393, 0.1352812796831131, 0.11173629760742188, 0.0921364352107048, 0.07443871349096298, 0.056015271693468094, 0.053058989346027374, 0.05338900908827782, 0.03812330216169357, 0.03889406844973564, 0.039090245962142944, 0.02917722426354885, 0.023007109761238098, 0.02968238666653633, 0.01784706488251686, 0.02592192403972149, 0.012253464199602604, 0.006347483489662409, 0.011432798579335213, 0.0188111774623394, 0.005072361323982477, 0.00046342608402483165, 0.0002334812597837299, 0.00012097381113562733, 9.602740465197712e-05, 7.931327127153054e-05, 6.616660539293662e-05, 6.393602961907163e-05, 5.559435885515995e-05]
100/100 [==============================] - 2s 18ms/step - loss: 1.0602 - accuracy: 0.8609
##  loss and accuracy ##
[1.060167

In [93]:
# import matplotlib.pyplot as plt
# acc = history.history['acc']
# val_acc = history.history['val_acc']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(len(acc))

# plt.plot(epochs, acc, 'bo', label='Training acc')
# plt.plot(epochs, val_acc, 'b', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.legend()

# plt.figure()

# plt.plot(epochs, loss, 'bo', label='Training loss')
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend()

# plt.show()

KeyError: ignored